In [1]:
import itertools
import sys
from pathlib import Path
from collections import defaultdict

from generate_all_functions import normalize_function, normalized_truth_tables, get_grouped_functions, get_all_functions

In [2]:
inputs = 3
outputs = 3
basis = 'bench'
circuits_dir = Path('./bench_res')

In [3]:
grouped_functions = get_grouped_functions(inputs, outputs, basis)

In [4]:
len(grouped_functions.keys())

60176

In [7]:
all_functions = set(get_all_functions(inputs, outputs))

In [8]:
len(all_functions)

341376

In [9]:
all_functions_from_grouped = set().union(*grouped_functions.values())

In [11]:
len(all_functions_from_grouped)

341376

In [12]:
get_class = {value: key for key, values in grouped_functions.items() for value in values}

In [13]:
len(get_class.keys())

341376

In [15]:
def check_availability(tables, circuits_dir):
    assert circuits_dir.exists()
    filename = '_'.join(tables) + '.txt'
    filepath = circuits_dir / filename
    return filepath.exists()

In [17]:
bad_set = set()
for function, normalized in get_class.items():
    if not check_availability(normalized, circuits_dir):
        bad_set.add(function)

In [20]:
len(bad_set)

45428

In [2]:
def get_function_size(tables, circuits_dir: Path) -> int:
    assert circuits_dir.exists()
    filename = '_'.join(tables) + '.txt'
    filepath = circuits_dir / filename
    if not filepath.exists():
        return -1  # Function not found
    with filepath.open('r') as file:
        lines = file.readlines()
        function_size = len(lines) - 2
        assert function_size >= 0
        return function_size

In [3]:
inputs = 3
outputs = 3
basis = 'bench'
circuits_dir = Path('./bench_res')

In [4]:
all_classes = list(normalized_truth_tables(inputs, outputs, basis))

In [5]:
len(all_classes)

101940

In [13]:
classes = defaultdict(int)

functions = defaultdict(int)

for i, tables in enumerate(itertools.combinations(itertools.product('01', repeat=2 ** inputs - 1), outputs)):
    tables = tuple(map(lambda x: '0' + ''.join(x), tables))
    # if len(set(tables)) != len(tables):
    #     continue
    class_representative = get_equivalence_class(inputs, tables, basis)
    # assert class_representative in all_classes, f"{class_representative}"
    function_size = get_function_size(class_representative, circuits_dir)
    if function_size == -1:
        print("Error")
        print(tables)
        print(class_representative)
        break
    classes[function_size] += int(tables == class_representative)
    functions[function_size] += 1 << outputs

    if i % 10000 == 0 and i > 0:
        print(f'============{i}============', file=sys.stderr)
        for s in sorted(classes.keys()):
            print(f'{s} -> {classes[s]}, {functions[s]}', file=sys.stderr)
        print(f'============{i}============', file=sys.stderr)

print("Final table:")
for s in sorted(classes.keys()):
    print(f'{s} -> {classes[s]}, {functions[s]}')

('00000000', '00010110', '00100000')
('00000000', '00010110', '00001000')
Final table:
3 -> 7, 184
4 -> 160, 3408
5 -> 445, 10416
6 -> 243, 5416
7 -> 7, 136


In [7]:
sanity_check = sum(functions.values()) == (2 ** (2 ** inputs)) ** outputs
print(f"Sanity check: {sanity_check}")
print(f"Total functions enumerated: {sum(functions.values())}")
print(f"{(2 ** (2 ** inputs)) ** outputs} functions expected")

Sanity check: True
Total functions enumerated: 16777216
16777216 functions expected


In [8]:
1 - classes[-1] / sum(classes.values())

0.9595617304868406

In [9]:
1 - functions[-1] / sum(functions.values())

0.8467140197753906